In [187]:
import numpy as np
import os
from scipy import signal
#from sp import multirate

In [238]:
# some variables that might be passed to this function or just declared globally:
EPHYS = {};
EPHYS['data'] =  np.random.rand(10000,10,16);
EPHYS['labels'] = range(16);
winsigma=.0025;
noise='none';
car_exclude= [];
savedir=os.getcwd();
min_f=1;
max_f=10e3;
hist_colors='jet';
mua_colors='hot';
figtitle='';
freq_range=[500, 4.5e3];
proc_fs=10e3;
downsampling=2;
channels=range(16);
hampel=6;
attenuation=40;
ripple=0.2;
car_trim=40;
fs = 30e3;
    #some stuff

In [278]:
import h5py
test_file = '/Users/Guitchounts/Dropbox (coxlab)/Ephys/Data/grat09/2015-10-05_11-57-24/experiment10_b/fxd_experiment10_100.raw.kwd'
input_h5file = h5py.File(test_file,'a')
data = np.double(input_h5file['/recordings/0/data'])

# shape should be (270000, 100,16) - samples x trials x channels 

data = data.reshape(((27000000/100,16,100))); # reshape from [samples x channels] to make 100 trials
data = data.transpose(0,2,1)


In [433]:
from scipy.stats import tmean, scoreatpercentile
def trimmean(arr, percent):
    # tmean should be avg across channels on a given trial. shape = [nsamples x ntrials]'
    lower_limit = scoreatpercentile(arr, percent/2)
    upper_limit = scoreatpercentile(arr, 100-percent/2)
    return tmean(arr, limits=(lower_limit, upper_limit), inclusive=(False, False))

In [397]:
def ephys_denoise_signal(data,chin,chout=[],method='none',car_exclude=[],car_trim=40):
    # Do you want to remove some channels from the analysis? Do you want to do CAR?
    CAR = [];
    if not chout: chout = chin;
    
    if car_exclude:
        car_electrodes = list(set(chin) - set(car_exclude))
        print 'Exluding channels %d from CAR' % car_electrodes
        print car_electrodes
    else:
        print 'Using all channels for CAR'
        car_electrodes = chin;
    
    [samples,ntrials,nchannels] = data.shape;
    
    if method == 'car':
        #trimmed mean to avoid artifacts and spikes:
        print 'Using electrodes ' + str(car_electrodes) + ' for CAR';
        print 'Trimmed mean prctile ' +  str(car_trim);
        
        CAR = np.empty(EPHYS['data'].shape)
        
        
        for elec in car_electrodes:         
            CAR[:,:,elec] = trimmean(data[:,:,car_electrodes],car_trim)
        
            data[:,:,elec] = data[:,:,elec] - CAR[:,:,elec];
        
    elif method == 'cmr': # common median reference ? 
        # other stuff
        pass
    
    
    return data, CAR

In [462]:
# test data set
x = data[0:10000,0:10,:]
x.shape

(10000, 10, 16)

(1000, 100)

In [463]:
tic = time.clock()
[nsamples,ntrials,nchannels] = x.shape;
trim_x = np.empty([nsamples,ntrials])
#for sample in range(nsamples):
#    for trial in range(ntrials):
#        trim_x[sample,trial] = trimmean(x,40)

np.apply_along_axis(trimmean, 2, x,40)

toc = time.clock()
toc - tic  
#trim_x = [trimmean(x[:,trial,ch],40) for ch in range(16),trial in range(100)]
#trim_x.shape

31.807186999999885

In [449]:
trim_x.shape

(100, 10)

In [380]:
def serial(arr, percent):
    #return [parzen_estimation(samples, x, w) for w in widths]
    return [trimmean(arr[:,:,elec],percent) for elec in range(16)] # replace range w/ car_electrodes


In [381]:
tic= time.clock()
result = serial(x,40)
toc = time.clock()
toc- tic
result

[38.068137475611586,
 37.339323520096023,
 36.11345259332122,
 35.667828145579435,
 38.710457865074865,
 37.908651441906983,
 36.369256683510947,
 37.273459383753504,
 43.013160089401872,
 42.41692507752326,
 39.915691999666308,
 38.027841685839086,
 41.505768397185825,
 39.51065390643236,
 37.417029438745729,
 38.067957441131348]

In [ ]:
def multiprocess(processes, arr, percent):
    pool = mp.Pool(processes=processes)
    results = [pool.apply_async(trimmean, args=(samples, x, w)) for w in widths]
    results = [p.get() for p in results]
    results.sort() # to sort the results by input window width
    return results

In [345]:
tic = time.clock()
data,CAR = ephys_denoise_signal(data,EPHYS['labels'],[],'car')
toc = time.clock()
toc - tic

Using all channels for CAR
Using electrodes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] for CAR
Trimmed mean prctile 40


SystemError: NULL result without error in PyObject_Call

In [335]:
import multiprocessing as mp
import time



def cube(x,y):
    return np.mean(x) +y


In [330]:
def serial(x, y):
    return [cube(x, y)]

def multiprocess(processes, x, y):
    pool = mp.Pool(processes=processes)
    results = [pool.apply_async(cube, args=(x, y))]
    results = [p.get() for p in results]
    results.sort() # to sort the results by input window width
    return results

In [336]:
results = multiprocess(4, range(10),3)
results

[7.5]

In [337]:
xxx = serial(range(10),3)
xxx

[7.5]

7.0

In [241]:
def ephys_condition_signal(data,freq_range=[500, 5e3],filt_name='b'):    
    medfilt_scale=[]; # median filter scale (in ms)
    filt_order=2;   
    filt_name='e'; # default to Butterworth, use a Kaiser filter if we need sharp cutoffs, per Logothetis et al. 2001
    ripple=.2; # Kaiser params, ripple in dB (linear in ellip)
    attenuation=40; # Kaiser params, attenuation (linear, dB in ellip)
    decomp_level=7;
    filt_type='bandpass';
    winsigma=.0025; # winsigma for Gaussian smoothing kernel (in s)

    
    [nsamples,ntrials,nchannels]=data.shape;
    
        
        
    # filtering:
    if filt_name == 'b': # Butterworth:
        print 'Butterworth Filter...'
        
        b,a= signal.butter(filt_order,[freq_range]/(fs/2),filt_type);
       #  b, a = signal.butter(4, 100, 'low', analog=True)
        
    elif filt_name == 'e': # Elliptical filter
        pass
        
    elif filt_name == 'k': # Kaiser
        pass
    elif filt_name == 'w': # Wavelet...
        pass
    else:
        print 'Please select an appropriate filter name'
        
    # rectify:
    print 'Rectifying Data...'
    data = data ** 2
    
    
    # smooth:
    print 'Smoothing data with Gaussian kernel. winsigma = ' + str(winsigma)
    
    edges = np.arange(-3*winsigma, 3*winsigma + 1/fs, 1/fs) # adding one 1/fs to get one more element (451 instead of 450)
    kernel=(1/(winsigma*np.sqrt(2*np.pi)))*np.exp((-edges**2)/(2*winsigma**2));

    kernel=kernel/sum(kernel); # normalize to sum to 1

    for ch in range(nchannels):
        for trial in range(ntrials):
            data[:,trial,ch] = np.convolve(data[:,trial,ch],kernel,'same');

    
    return data

In [242]:
data = ephys_condition_signal(EPHYS['data'])

Rectifying Data...
Smoothing data with Gaussian kernel. winsigma = 0.0025


7.7715611723760958e-16

In [31]:

################################################################

#################### SIGNAL CONDITIONING ################################

downfact = fs / proc_fs;

if downfact%1>0:
	print 'Downsample factor must be integer'
	# should raise an error...


proc_data = ephys_denoise_signal(EPHYS['data'],EPHYS['labels'],channels,noise,car_exclude, car_trim);

print 'Anti-alias filtering and downsampling';


proc_data = ephys_condition_signal(proc_data,[300, proc_fs/2]);

print 'Downsampling to ' + str(proc_fs);
proc_data = proc_data[0::downfact]; # figure out best way to do this!
# why does proc_data get passed to ephys_condition_signal again? line 149. Make it a float32 (single) also




[nsamples, ntrials,nchannels] = proc_data.shape # ? is proc_data an np array?
TIME = range(nsamples)/ proc_fs;

# "if downsampling:"
print 'Downsampling by factor of ' + str(downfact);
MUA = {};
MUA['t'] = TIME[0::downsampling];
MUA['image'] = np.zeros(shape=(ntrials,len(MUA['t']),len(channels)),dtype='float32')

for i in range(nchannels): MUA['image'][:,:,i] = proc_data[0::downsampling,:,i]; # this was transposed in .m - why?

MUA['channels'] = channels;
MUA['trials'] = trials;

################################################################################

print 'Generating figures....';
print 'Saving to directory ' + str(savedir);

[path, name,ext] = os.path.split(savedir) # what is ext? need to get rid of it
# or just get pwd and make mua folder there




Anti-alias filtering and downsampling
Downsampling to 10000.0


TypeError: 'NoneType' object has no attribute '__getitem__'